<a href="https://colab.research.google.com/github/vasanththangavel/Web_Map/blob/master/earth_engine_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install earthengine-api
!pip install folium

In [2]:
import ee

In [3]:
## Trigger the authentication flow. You only need to do this once
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=v3ftDCgRefoladqAQusiatD9gEC8STRj8pmaBVVblxU&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g4zSBwjJcsCqctN4QsN_0zcB6lwS16hCV_Ayu6f2ExzKIupYJk59fY

Successfully saved authorization token.


In [10]:
# dem=ee.ImageCollection("COPERNICUS/S2_SR")
# point_coords=ee.Geometry.Point(11.2485,78.3387)
# elev = dem.sample(point_coords,30)
# print(dem)
# print("Kolli hills elevation in m",elev.first().get('elevation').getInfo())

ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "ImageCollection.load",
    "arguments": {
      "id": {
        "constantValue": "COPERNICUS/S2_SR"
      }
    }
  }
})


In [28]:
import folium
from IPython.display import Image

map=folium.Map(location=[37.93418,-101.05259],zoom_start=13,tiles='Stamen Terrain',height=500,width=1000)
-106.06, 37.71

tile_layer=folium.TileLayer(tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
                                attr = 'Google',
                                name = 'Google Satellite',
                                overlay = True,
                                control = True)

def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))


image = ee.Image('LANDSAT/LT05/C01/T1_TOA/LT05_031034_20110619')
ndvi = image.normalizedDifference(['B4','B3'])
ndwi = image.normalizedDifference(['B3', 'B5'])
# Detect edges in the composite.
canny = ee.Algorithms.CannyEdgeDetector(ndvi, 0.7)
# Mask the image with itself to get rid of areas with no edges.
canny = canny.updateMask(canny)
tile_layer.add_to(map)
palette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
  '74A901', '66A000', '529400', '3E8601', '207401', '056201',
  '004C00', '023B01', '012E01', '011D01', '011301']
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['FF0000']}

ndvi_vis_params = {
'min': 0,
'max': 1,'palette':palette}
ndwi_vis_params = {'min': 0.5, 
 'max': 1, 
 'palette': ['00FFFF', '0000FF']}

folium.Map.add_ee_layer = add_ee_layer

map.add_ee_layer(canny, vis_params, 'Canny Edges')
map.add_ee_layer(ndvi, ndvi_vis_params, 'Landsat NDVI')
map.add_ee_layer(ndvi, ndwi_vis_params, 'Landsat NDWI')
folium.Marker([37.93418,-101.05259],popup='NDVI & NDWI Index',tooltip='click here to see popup').add_to(map)
map.add_child(folium.LayerControl())
display(map)


In [41]:
new_image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_034034_20170608')
new_ndvi = new_image.normalizedDifference(['B5','B4'])
new_map=folium.Map(location=[37.71,-106.06],zoom_start=13,tiles='Stamen Terrain',height=500,width=1000)
new_map.add_ee_layer(new_ndvi, ndvi_vis_params, 'Landsat New_NDVI')
farmSize = 500

circleKernel = ee.Kernel.circle(farmSize, 'meters')
squareKernel = ee.Kernel.square(farmSize, 'meters')
circles = new_ndvi.convolve(circleKernel)
squares = new_ndvi.convolve(squareKernel)
diff = circles.subtract(squares)

diff = diff.abs().multiply(100).toByte()
max = diff.focal_max(farmSize * 1.8,'meters')

local = diff.where(diff.neq(max), 0)
thresh = local.gt(2)


In [42]:
peaks = thresh.focal_max(circleKernel)
new_map.add_ee_layer(peaks.updateMask(peaks), {'palette': 'FF3737'}, 'Kernel Peaks');

# Detect the edges of the features.  Discard the edges with lower intensity.
new_canny = ee.Algorithms.CannyEdgeDetector(new_ndvi, 0)
new_canny = new_canny.gt(0.3)

# Create a "ring" kernel from two circular kernels.
inner = ee.Kernel.circle(farmSize - 20, 'meters', False, -1)
outer = ee.Kernel.circle(farmSize + 20, 'meters', False, 1)
ring = outer.add(inner, True)

# Highlight the places where the feature edges best match the circle kernel.
centers = new_canny.convolve(ring).gt(0.5).focal_max(circleKernel)
new_map.add_ee_layer(centers.updateMask(centers), {'palette': '4285FF'}, 'Ring centers')
display(new_map)

Could not display Kernel Peaks
Could not display Ring centers
